In [1]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

# Set paths
image_dir = "dataset/images"
annotation_dir = "dataset/annotations"
output_dir = "dataset"

labels_map = {
    "with_mask": "with_mask",
    "without_mask": "without_mask"
}

os.makedirs(f"{output_dir}/with_mask", exist_ok=True)
os.makedirs(f"{output_dir}/without_mask", exist_ok=True)

# Loop through annotation files
for xml_file in os.listdir(annotation_dir):
    if not xml_file.endswith(".xml"):
        continue

    tree = ET.parse(os.path.join(annotation_dir, xml_file))
    root = tree.getroot()
    filename = root.find('filename').text
    img_path = os.path.join(image_dir, filename)

    # Open image
    img = Image.open(img_path)

    for obj in root.findall('object'):
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(float(bbox.find('xmin').text))
        ymin = int(float(bbox.find('ymin').text))
        xmax = int(float(bbox.find('xmax').text))
        ymax = int(float(bbox.find('ymax').text))

        cropped_img = img.crop((xmin, ymin, xmax, ymax))
        resized_img = cropped_img.resize((128, 128))

        # Save to folder
        label_folder = labels_map.get(label)
        if label_folder:
            save_path = os.path.join(output_dir, label_folder, f"{filename}_{xmin}_{ymin}.jpg")
            resized_img.convert("RGB").save(save_path)



## Step 3.1: Load the dataset using ImageDataGenerator

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = "dataset/processed_dataset"

datagen = ImageDataGenerator(
    rescale=1./255,         # Normalize images
    validation_split=0.2,   # 80% train, 20% val
    horizontal_flip=True,   # Basic augmentation
    zoom_range=0.2,
    shear_range=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3151 images belonging to 2 classes.
Found 786 images belonging to 2 classes.


## Step 4: Build the Model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')  # 2 classes: with_mask, without_mask
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

c:\Users\Black\miniconda3\envs\ml_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,578 (28.20 MB)

 Trainable params: 7,392,578 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

## Step 5: Train the Model

In [4]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

c:\Users\Black\miniconda3\envs\ml_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 98s 905ms/step - accuracy: 0.8420 - loss: 0.5435 - val_accuracy: 0.9593 - val_loss: 0.1220
Epoch 2/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 72s 723ms/step - accuracy: 0.9632 - loss: 0.1116 - val_accuracy: 0.9567 - val_loss: 0.1198
Epoch 3/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 58s 584ms/step - accuracy: 0.9604 - loss: 0.1065 - val_accuracy: 0.9555 - val_loss: 0.1367
Epoch 4/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 78s 544ms/step - accuracy: 0.9637 - loss: 0.1085 - val_accuracy: 0.9529 - val_loss: 0.1768
Epoch 5/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 56s 567ms/step - accuracy: 0.9631 - loss: 0.1128 - val_accuracy: 0.9618 - val_loss: 0.1024
Epoch 6/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 55s 558ms/step - accuracy: 0.9661 - loss: 0.0868 - val_accuracy: 0.9720 - val_loss: 0.0863
Epoch 7/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 55s 553ms/step - accuracy: 0.9749 - loss: 0.0765 - val_accuracy: 0.9555 - val_loss: 0.1147
Epoch 8/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 54s 545ms/step - accuracy: 0.9747 - loss: 0.0798 - val_accu

In [5]:
import os

print(os.listdir('dataset'))

['annotations', 'images', 'processed_dataset', 'without_mask', 'with_mask']


In [6]:
print(train_generator.class_indices)

{'with_mask': 0, 'without_mask': 1}
